In [ ]:
import subprocess, time

subjects = list(range(1, 10))
gpus = [0, 1]
max_per_gpu = 2  # ⭐ 控制这里

running = {0: [], 1: []}

for subj in subjects:
    # 找当前最空的 GPU
    gpu = min(running, key=lambda k: len(running[k]))

    # 等待 GPU 空位
    while len(running[gpu]) >= max_per_gpu:
        time.sleep(5)
        running[gpu] = [p for p in running[gpu] if p.poll() is None]

    p = subprocess.Popen([
        "python", "train_kfold_script.py",
        "--subject", str(subj),
        "--gpu", str(gpu)
    ])
    running[gpu].append(p)

# 等待全部结束
for plist in running.values():
    for p in plist:
        p.wait()


In [ ]:
import glob, numpy as np

epochs = []
for f in glob.glob("results/subj_*.txt"):
    epochs.append(int(open(f).read()))

median_epoch = int(np.median(epochs))
print("Median Epoch:", median_epoch)